# 數位韌性指數計算系統 - 優化版

## 優化重點
1. **集中配置管理**：所有關鍵參數集中在開頭
2. **緩存機制**：避免重複計算和 API 調用
3. **主題層級評分**：Phase 4 基於主題而非文本評分，大幅提升速度
4. **多輪迭代優化**：Phase 3 支持多輪優化
5. **關鍵詞增強提示**：LLM 映射時提供主題關鍵詞

In [1]:
# ========================================
# 全局配置區 - 所有重要設置集中管理
# ========================================

from pathlib import Path
import json

# === API 配置 ===
from dotenv import dotenv_values
config = dotenv_values(".env")
OPENAI_API_KEY = config.get("OPENAI_API_KEY")

# === 模型配置 ===
EMBEDDING_MODEL = 'text-embedding-3-small'  # OpenAI 嵌入模型
LLM_MODEL = 'gpt-5-nano-2025-08-07'  # LLM 模型（用於主題優化和評分）
LLM_TEMPERATURE = 1  # 溫度參數（降低隨機性）

# === 文件路徑配置 ===
DATA_DIR = Path('data')
CORPUS_PATH = DATA_DIR / 'corpus.csv'

# Phase 2 輸出
EMBEDDINGS_PATH = DATA_DIR / 'embeddings_text-3-small.npy'
EMBEDDINGS_INDEX_PATH = DATA_DIR / 'embeddings_index.json'
PHASE2_MODEL_DIR = DATA_DIR / 'part2_bertopic_model'
PHASE2_TOPICS_CSV = DATA_DIR / 'part2_topics.csv'
PHASE2_DOC_PROBS = DATA_DIR / 'part2_doc_topic_probs.npy'
PHASE2_CORPUS_CSV = DATA_DIR / 'part2_corpus_with_topics.csv'
PHASE2_TOPIC_YEAR_CSV = DATA_DIR / 'part2_topic_prop_by_year.csv'

# Phase 3 輸出
PHASE3_MODEL_DIR = DATA_DIR / 'part3_optimized_bertopic_model'
PHASE3_CORPUS_CSV = DATA_DIR / 'part3_corpus_with_topics_v2.csv'
PHASE3_OPTIMIZATION_CACHE = DATA_DIR / 'phase3_optimization_plans.json'

# Phase 4 輸出
PHASE4_TOPIC_DIM_MAP_CACHE = DATA_DIR / 'phase4_topic_dimension_map.json'
PHASE4_TOPIC_SCORES_CACHE = DATA_DIR / 'phase4_topic_dimension_scores.json'
PHASE4_DOC_SCORES_CSV = DATA_DIR / 'part4_doc_dimension_scores.csv'
PHASE4_DRI_CSV = DATA_DIR / 'part4_entity_time_dri.csv'

# === BERTopic 參數配置 ===
# Phase 2 初始參數
UMAP_N_NEIGHBORS = 15
UMAP_N_COMPONENTS = 10
UMAP_MIN_DIST = 0.0
UMAP_METRIC = 'cosine'
HDBSCAN_MIN_CLUSTER_SIZE = 30
HDBSCAN_MIN_SAMPLES = 10
HDBSCAN_METRIC = 'euclidean'
HDBSCAN_SELECTION_METHOD = 'eom'

# === Phase 3 優化配置 ===
OPTIMIZATION_ITERATIONS = 2  # LLM 迭代優化次數（建議 2-3 次）
TOPIC_SAMPLE_SIZE = 3  # 每個主題採樣的例句數量

# === Phase 4 評分配置 ===
DIMENSIONS = ["ITC", "ACAP", "DC", "GOVSEC", "DATA", "ECO", "OTHER"]  # 數位韌性構面
DIMENSION_WEIGHTS = {  # 各構面權重（總和為 1）
    "ITC": 0.20,
    "ACAP": 0.20,
    "DC": 0.15,
    "GOVSEC": 0.15,
    "DATA": 0.15,
    "ECO": 0.15,
    "OTHER": 0.0
}

# 評分規則
SCORING_RUBRIC = (
    "Rate the substantiveness and strength on a 0–5 scale:\n"
    "0 = irrelevant/very vague\n"
    "3 = part of a specific action or quantitative indicator\n"
    "5 = clear, quantitative, auditable, and directly related to strategy/investment/institutionalization\n"
    "Output JSON: {\"score\": number, \"reasoning\": \"brief explanation\"}"
)

# === 其他配置 ===
EMBEDDING_BATCH_SIZE = 256
RANDOM_SEED = 42

print("✓ 配置加載完成")
print(f"  - 嵌入模型: {EMBEDDING_MODEL}")
print(f"  - LLM 模型: {LLM_MODEL}")
print(f"  - 數據目錄: {DATA_DIR}")
print(f"  - Phase 3 迭代次數: {OPTIMIZATION_ITERATIONS}")
print(f"  - 評分構面: {', '.join(DIMENSIONS)}")

✓ 配置加載完成
  - 嵌入模型: text-embedding-3-small
  - LLM 模型: gpt-5-nano-2025-08-07
  - 數據目錄: data
  - Phase 3 迭代次數: 2
  - 評分構面: ITC, ACAP, DC, GOVSEC, DATA, ECO, OTHER


In [2]:
# ========================================
# 導入必要的庫
# ========================================

import os
import json
import re
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from typing import Dict, List, Tuple

# OpenAI
from openai import OpenAI

# BERTopic and dependencies
from bertopic import BERTopic
from umap import UMAP
import hdbscan

# Sklearn
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

# 初始化 OpenAI 客戶端
client = OpenAI(api_key=OPENAI_API_KEY)

# 確保數據目錄存在
DATA_DIR.mkdir(exist_ok=True)

print("✓ 庫導入完成")

✓ 庫導入完成


---
# Phase 0: 數據下載與預處理
（此部分保持不變，如需重新下載請取消註釋）

In [3]:
# from src.data_download import download_sec_filings
# from src.data_processing import process_sec_filings

# download_sec_filings()
# process_sec_filings(config)

---
# Phase 2: 初始主題生成（BERTopic）

- 嵌入模型：OpenAI text-embedding-3-small (1536維)
- 降維：UMAP，聚類：HDBSCAN
- **緩存機制**：嵌入向量緩存，避免重複計算

In [4]:
# ========================================
# 加載語料庫
# ========================================

assert CORPUS_PATH.exists(), f"找不到語料庫: {CORPUS_PATH}"

df = pd.read_csv(CORPUS_PATH)
df.columns = [c.strip().lower() for c in df.columns]
assert 'text' in df.columns, "語料需要包含 'text' 欄位"

# 偵測元數據欄位
meta_cols = [c for c in ['doc_id', 'company', 'firm', 'ticker', 'year', 'date'] if c in df.columns]
print(f"✓ 載入語料庫: {len(df)} 筆文檔")
print(f"  - 元數據欄位: {meta_cols or '(無)'}")

texts = df['text'].astype(str).tolist()

✓ 載入語料庫: 6233 筆文檔
  - 元數據欄位: ['ticker', 'year']


In [5]:
# ========================================
# 生成/載入嵌入向量（帶緩存）
# ========================================

if EMBEDDINGS_PATH.exists() and EMBEDDINGS_INDEX_PATH.exists():
    print("✓ 從緩存加載嵌入向量...")
    embeddings = np.load(EMBEDDINGS_PATH)
    with open(EMBEDDINGS_INDEX_PATH, 'r') as f:
        emb_info = json.load(f)
    print(f"  - 模型: {emb_info.get('model')}")
    print(f"  - 數量: {emb_info.get('count')}")
else:
    print("⚙ 生成嵌入向量（這可能需要幾分鐘）...")
    vecs = []
    for i in tqdm(range(0, len(texts), EMBEDDING_BATCH_SIZE), desc="生成嵌入"):
        batch = texts[i:i + EMBEDDING_BATCH_SIZE]
        response = client.embeddings.create(model=EMBEDDING_MODEL, input=batch)
        vecs.extend([np.array(d.embedding, dtype=np.float32) for d in response.data])
    
    embeddings = np.vstack(vecs)
    np.save(EMBEDDINGS_PATH, embeddings)
    
    with open(EMBEDDINGS_INDEX_PATH, 'w') as f:
        json.dump({'count': len(texts), 'model': EMBEDDING_MODEL}, f)
    
    print(f"✓ 嵌入向量已保存")

print(f"  - 嵌入形狀: {embeddings.shape}")

✓ 從緩存加載嵌入向量...
  - 模型: text-embedding-3-small
  - 數量: 6233
  - 嵌入形狀: (6233, 1536)


In [6]:
# ========================================
# 初始 BERTopic 模型訓練
# ========================================

print("⚙ 訓練 BERTopic 模型...")

umap_model = UMAP(
    n_neighbors=UMAP_N_NEIGHBORS,
    n_components=UMAP_N_COMPONENTS,
    min_dist=UMAP_MIN_DIST,
    metric=UMAP_METRIC,
    random_state=RANDOM_SEED
)

hdbscan_model = hdbscan.HDBSCAN(
    min_cluster_size=HDBSCAN_MIN_CLUSTER_SIZE,
    min_samples=HDBSCAN_MIN_SAMPLES,
    metric=HDBSCAN_METRIC,
    cluster_selection_method=HDBSCAN_SELECTION_METHOD,
    prediction_data=True
)

topic_model = BERTopic(
    calculate_probabilities=True,
    verbose=True,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model
)

topics, probs = topic_model.fit_transform(texts, embeddings=embeddings)
df['topic'] = topics

# 保存結果
topic_model.save(PHASE2_MODEL_DIR.as_posix(), serialization="safetensors")
topic_info = topic_model.get_topic_info()
topic_info.to_csv(PHASE2_TOPICS_CSV, index=False, encoding='utf-8')
if probs is not None:
    np.save(PHASE2_DOC_PROBS, probs)
df.to_csv(PHASE2_CORPUS_CSV, index=False, encoding='utf-8')

# 按年度分析主題分佈
if 'year' in df.columns:
    year_dist = df.groupby('year')['topic'].value_counts(normalize=True).rename('prop').reset_index()
    year_dist.to_csv(PHASE2_TOPIC_YEAR_CSV, index=False, encoding='utf-8')

print("✓ Phase 2 完成")
print(f"  - 主題數量: {len(topic_info[topic_info['Topic'] != -1])}")
print(f"  - 離群點比例: {(np.array(topics) == -1).mean():.2%}")
topic_info.head(10)

2025-10-01 01:39:48,273 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm


⚙ 訓練 BERTopic 模型...


2025-10-01 01:39:58,695 - BERTopic - Dimensionality - Completed ✓
2025-10-01 01:39:58,695 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-10-01 01:39:59,170 - BERTopic - Cluster - Completed ✓
2025-10-01 01:39:59,172 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-10-01 01:39:59,668 - BERTopic - Representation - Completed ✓


✓ Phase 2 完成
  - 主題數量: 65
  - 離群點比例: 14.76%


,Topic,Count,Name,Representation,Representative_Docs
0,-1,920,-1_our_of_and_the,"[our, of, and, the, to, or, in, we, as, on]",[Our Supplier Relationships Subject Us to a Nu...
1,0,314,0_tax_income_taxes_deferred,"[tax, income, taxes, deferred, foreign, the, r...",[We record provision for income taxes for the ...
2,1,205,1_gas_production_oil_reserves,"[gas, production, oil, reserves, proved, exxon...",[These excluded amounts for both\nconsolidated...
3,2,203,2_currency_foreign_derivative_exchange,"[currency, foreign, derivative, exchange, hedg...",[A master netting arrangement allows counterpa...
4,3,162,3_goodwill_assets_impairment_intangible,"[goodwill, assets, impairment, intangible, val...","[Upon the occurrence of a triggering event, we..."
5,4,154,4_capital_basel_bank_liquidity,"[capital, basel, bank, liquidity, reserve, fed...",[banks are subject to quantitative and qualita...
6,5,153,5_our_may_we_could,"[our, may, we, could, or, products, results, a...",[Demand for our products is variable and hard ...
7,6,152,6_driven_banking_income_higher,"[driven, banking, income, higher, net, fees, r...","[Net revenue was\n$52.1 billion\n, an increase..."
8,7,150,7_products_product_or_our,"[products, product, or, our, healthcare, to, a...",[presidential administration’s policy proposal...
9,8,137,8_health_care_medicare_medical,"[health, care, medicare, medical, unitedhealth...",[UnitedHealthcare Medicare & Retirement provid...


---
# Phase 3: LLM 迭代優化（多輪自我批改）

**優化重點**：
1. 支持多輪迭代（配置中設定次數）
2. 優化計劃緩存，避免重複 LLM 調用
3. 每輪追蹤指標變化

In [7]:
# ========================================
# 工具函數：指標計算
# ========================================

def compute_topic_centers(emb: np.ndarray, topics: List[int]) -> Dict[int, np.ndarray]:
    """計算每個主題的中心向量"""
    centers = {}
    s = pd.Series(topics)
    for tid, idxs in s.groupby(s).groups.items():
        if tid == -1:
            continue
        vecs = emb[list(idxs)]
        centers[tid] = normalize(vecs.mean(axis=0, keepdims=True))[0]
    return centers

def compute_metrics(emb: np.ndarray, topics: List[int]) -> Tuple[Dict, float, float, float]:
    """計算主題質量指標：一致性、區分度、輪廓係數、離群率"""
    centers = compute_topic_centers(emb, topics)
    
    # 一致性（Cohesion）：每個主題內部的平均相似度
    s = pd.Series(topics)
    cohesion = {}
    for tid, idxs in s.groupby(s).groups.items():
        if tid == -1 or tid not in centers:
            continue
        sims = cosine_similarity(emb[list(idxs)], centers[tid].reshape(1, -1)).ravel()
        cohesion[tid] = float(np.mean(sims))
    
    # 區分度（Separation）：主題中心之間的平均距離
    separation = np.nan
    if len(centers) >= 2:
        center_matrix = np.vstack(list(centers.values()))
        separation = cosine_distances(center_matrix).mean()
    
    # 輪廓係數（Silhouette）
    mask = np.array(topics) != -1
    silhouette = np.nan
    if mask.sum() > 5 and len(set(np.array(topics)[mask])) > 1:
        silhouette = silhouette_score(emb[mask], np.array(topics)[mask])
    
    # 離群率
    outlier_rate = (np.array(topics) == -1).mean()
    
    return cohesion, separation, silhouette, outlier_rate

def print_metrics(cohesion: Dict, separation: float, silhouette: float, outlier: float, prefix=""):
    """打印指標"""
    coh_mean = np.mean(list(cohesion.values())) if cohesion else np.nan
    print(f"{prefix}一致性: {coh_mean:.4f}")
    print(f"{prefix}區分度: {separation:.4f}")
    print(f"{prefix}Silhouette: {silhouette:.4f}")
    print(f"{prefix}離群率: {outlier:.2%}")

print("✓ 工具函數就緒")

✓ 工具函數就緒


In [8]:
# ========================================
# 迭代優化主循環
# ========================================

# 載入 Phase 2 結果
df = pd.read_csv(PHASE2_CORPUS_CSV)
embeddings = np.load(EMBEDDINGS_PATH)
current_model = BERTopic.load(PHASE2_MODEL_DIR.as_posix())
current_topics = df['topic'].tolist()

print(f"⚙ 開始 {OPTIMIZATION_ITERATIONS} 輪迭代優化...\n")

# 記錄優化歷史
optimization_history = []

# 初始指標
coh, sep, sil, out = compute_metrics(embeddings, current_topics)
print("初始指標:")
print_metrics(coh, sep, sil, out, "  ")
print()

for iteration in range(OPTIMIZATION_ITERATIONS):
    print(f"{'='*50}")
    print(f"第 {iteration + 1} 輪優化")
    print(f"{'='*50}")
    
    # 1. 採樣主題代表詞和例句
    def sample_topic_info(model, df, topic_col='topic', k=TOPIC_SAMPLE_SIZE):
        samples = {}
        for tid in model.get_topic_info()['Topic'].tolist():
            if tid == -1:
                continue
            words = ', '.join([w for w, _ in model.get_topic(tid)[:10]])
            examples = df[df[topic_col] == tid]['text'].head(k).tolist()
            samples[tid] = {"words": words, "examples": examples}
        return samples
    
    samples = sample_topic_info(current_model, df, 'topic' if iteration == 0 else f'topic_v{iteration+1}')
    
    # 2. 請 LLM 給出優化建議
    print("⚙ 請求 LLM 優化建議...")
    
    plan_prompt = (
        "You are a topic modeling expert. Analyze the topics below and provide optimization suggestions.\n"
        "Output JSON with:\n"
        "- 'merge_pairs': [[topic_a, topic_b], ...] - topics to merge\n"
        "- 'split_topics': [id, ...] - topics that are too broad\n"
        "- 'rename': {id: 'descriptive name'} - better topic names\n"
        "- 'new_stopwords': ['word', ...] - common words to filter\n"
        "- 'params': {'min_cluster_size': int, 'min_samples': int, ...} - HDBSCAN/UMAP params\n"
        "Focus on improving cohesion, separation, and reducing outliers.\n"
        "Output ONLY valid JSON, no explanation."
    )
    
    response = client.chat.completions.create(
        model=LLM_MODEL,
        temperature=LLM_TEMPERATURE,
        messages=[
            {"role": "system", "content": "You are a topic modeling optimization expert. Output JSON only."},
            {"role": "user", "content": plan_prompt},
            {"role": "user", "content": json.dumps({"topics": samples}, ensure_ascii=False)}
        ]
    )
    
    raw = response.choices[0].message.content
    try:
        plan = json.loads(raw)
    except Exception:
        # 嘗試提取 JSON
        match = re.search(r'\{[\s\S]*\}', raw)
        if match:
            plan = json.loads(match.group(0))
        else:
            print("⚠ 無法解析 LLM 輸出，跳過本輪")
            continue
    
    print(f"  - 合併建議: {len(plan.get('merge_pairs', []))} 對")
    print(f"  - 拆分建議: {len(plan.get('split_topics', []))} 個主題")
    print(f"  - 重命名: {len(plan.get('rename', {}))} 個主題")
    
    # 3. 應用參數優化
    print("⚙ 根據建議重新訓練模型...")
    
    params = plan.get('params', {})
    min_cluster_size = int(params.get('min_cluster_size', HDBSCAN_MIN_CLUSTER_SIZE))
    min_samples = int(params.get('min_samples', HDBSCAN_MIN_SAMPLES))
    n_neighbors = int(params.get('n_neighbors', UMAP_N_NEIGHBORS))
    n_components = int(params.get('n_components', UMAP_N_COMPONENTS))
    
    umap_opt = UMAP(
        n_neighbors=n_neighbors,
        n_components=n_components,
        min_dist=UMAP_MIN_DIST,
        metric=UMAP_METRIC,
        random_state=RANDOM_SEED
    )
    
    hdbscan_opt = hdbscan.HDBSCAN(
        min_cluster_size=min_cluster_size,
        min_samples=min_samples,
        metric=HDBSCAN_METRIC,
        cluster_selection_method=HDBSCAN_SELECTION_METHOD,
        prediction_data=True
    )
    
    optimized_model = BERTopic(
        calculate_probabilities=True,
        verbose=False,
        umap_model=umap_opt,
        hdbscan_model=hdbscan_opt
    )
    
    new_topics, new_probs = optimized_model.fit_transform(texts, embeddings=embeddings)
    
    # 4. 應用重命名
    if plan.get('rename') and isinstance(plan['rename'], dict):
        rename_map = {int(k): v for k, v in plan['rename'].items()}
        for tid, name in rename_map.items():
            try:
                optimized_model.set_topic_labels({tid: name})
            except:
                pass
    
    # 5. 計算新指標
    new_coh, new_sep, new_sil, new_out = compute_metrics(embeddings, new_topics)
    
    print("\n結果對比:")
    print_metrics(coh, sep, sil, out, "  舊: ")
    print_metrics(new_coh, new_sep, new_sil, new_out, "  新: ")
    
    # 6. 保存結果
    topic_col = f'topic_v{iteration + 2}'
    df[topic_col] = new_topics
    current_model = optimized_model
    current_topics = new_topics
    coh, sep, sil, out = new_coh, new_sep, new_sil, new_out
    
    # 記錄歷史（轉換為 Python 原生類型）
    optimization_history.append({
        'iteration': iteration + 1,
        'plan': plan,
        'cohesion': float(np.mean(list(new_coh.values()))) if new_coh else None,
        'separation': float(new_sep) if not np.isnan(new_sep) else None,
        'silhouette': float(new_sil) if not np.isnan(new_sil) else None,
        'outlier_rate': float(new_out)
    })
    
    print(f"\n✓ 第 {iteration + 1} 輪完成\n")

# 保存最終結果
current_model.save(PHASE3_MODEL_DIR.as_posix(), serialization="safetensors")
df.to_csv(PHASE3_CORPUS_CSV, index=False, encoding='utf-8')

# 保存優化歷史
with open(PHASE3_OPTIMIZATION_CACHE, 'w') as f:
    json.dump(optimization_history, f, indent=2, ensure_ascii=False)

print("="*50)
print("✓ Phase 3 完成")
print(f"  - 最終主題數: {len(set([t for t in current_topics if t != -1]))}")
print(f"  - 優化歷史已保存: {PHASE3_OPTIMIZATION_CACHE}")

2025-10-01 01:40:00,366 - BERTopic - WARNING: You are loading a BERTopic model without explicitly defining an embedding model. If you want to also load in an embedding model, make sure to use `BERTopic.load(my_model, embedding_model=my_embedding_model)`.


⚙ 開始 2 輪迭代優化...

初始指標:
  一致性: 0.7668
  區分度: 0.4119
  Silhouette: 0.0708
  離群率: 14.76%

第 1 輪優化
⚙ 請求 LLM 優化建議...
  - 合併建議: 8 對
  - 拆分建議: 6 個主題
  - 重命名: 65 個主題
⚙ 根據建議重新訓練模型...

結果對比:
  舊: 一致性: 0.7668
  舊: 區分度: 0.4119
  舊: Silhouette: 0.0708
  舊: 離群率: 14.76%
  新: 一致性: 0.8045
  新: 區分度: 0.4634
  新: Silhouette: 0.0758
  新: 離群率: 19.96%

✓ 第 1 輪完成

第 2 輪優化
⚙ 請求 LLM 優化建議...
  - 合併建議: 6 對
  - 拆分建議: 10 個主題
  - 重命名: 11 個主題
⚙ 根據建議重新訓練模型...

結果對比:
  舊: 一致性: 0.8045
  舊: 區分度: 0.4634
  舊: Silhouette: 0.0758
  舊: 離群率: 19.96%
  新: 一致性: 0.7958
  新: 區分度: 0.4569
  新: Silhouette: 0.0745
  新: 離群率: 17.21%

✓ 第 2 輪完成

✓ Phase 3 完成
  - 最終主題數: 117
  - 優化歷史已保存: data/phase3_optimization_plans.json


---
# Phase 4: 主題映射與評分（優化版）

**核心優化**：
1. **提供主題關鍵詞**：LLM 映射時不再是 "Topic 0"，而是 "Topic 0: tax, income, taxes, deferred..."
2. **主題層級評分**：先對每個主題評分，再根據文檔的主題分佈加權計算
3. **大幅加速**：從 6000+ 文檔×7 構面 → ~70 主題×7 構面

In [9]:
# ========================================
# 載入 Phase 3 結果
# ========================================

print("⚙ 載入優化後的主題模型...")

# 嘗試載入最新版本
if PHASE3_CORPUS_CSV.exists():
    df = pd.read_csv(PHASE3_CORPUS_CSV)
    model = BERTopic.load(PHASE3_MODEL_DIR.as_posix())
    # 找到最後一個 topic 欄位
    topic_cols = [c for c in df.columns if c.startswith('topic')]
    TOPIC_COL = topic_cols[-1] if topic_cols else 'topic'
    print(f"  - 使用主題欄位: {TOPIC_COL}")
else:
    print("  - Phase 3 結果未找到，使用 Phase 2")
    df = pd.read_csv(PHASE2_CORPUS_CSV)
    model = BERTopic.load(PHASE2_MODEL_DIR.as_posix())
    TOPIC_COL = 'topic'

df.columns = [c.strip().lower() for c in df.columns]
print(f"  - 文檔數: {len(df)}")
print(f"  - 主題數: {df[TOPIC_COL].nunique() - 1}")

2025-10-01 01:41:42,335 - BERTopic - WARNING: You are loading a BERTopic model without explicitly defining an embedding model. If you want to also load in an embedding model, make sure to use `BERTopic.load(my_model, embedding_model=my_embedding_model)`.


⚙ 載入優化後的主題模型...
  - 使用主題欄位: topic_v3
  - 文檔數: 6233
  - 主題數: 117


In [10]:
# ========================================
# Step 1: 主題 → 構面映射（帶關鍵詞）
# ========================================

print("⚙ 映射主題到數位韌性構面...")

# 檢查緩存
if PHASE4_TOPIC_DIM_MAP_CACHE.exists():
    print("  - 從緩存載入映射...")
    with open(PHASE4_TOPIC_DIM_MAP_CACHE, 'r') as f:
        topic_to_dimension = json.load(f)
    # 轉換 key 為 int
    topic_to_dimension = {int(k): v for k, v in topic_to_dimension.items()}
else:
    print("  - 生成新映射（請求 LLM）...")
    
    # 獲取所有主題及其關鍵詞
    topic_ids = sorted([int(t) for t in df[TOPIC_COL].dropna().unique() if t != -1])
    topic_descriptions = {}
    
    for tid in topic_ids:
        # 獲取主題的代表詞（前10個）
        try:
            words = [w for w, _ in model.get_topic(tid)[:10]]
            topic_descriptions[tid] = f"Topic {tid}: {', '.join(words)}"
        except:
            topic_descriptions[tid] = f"Topic {tid}"
    
    # 請求 LLM 映射
    mapping_prompt = (
        "You are a research assistant. Map each topic to ONE digital resilience dimension:\n"
        f"Dimensions: {', '.join(DIMENSIONS)}\n\n"
        "Dimension definitions:\n"
        "- ITC: IT infrastructure, cloud, networks, hardware, software systems\n"
        "- ACAP: Cybersecurity, threat detection, access control, encryption\n"
        "- DC: Data centers, disaster recovery, business continuity, redundancy\n"
        "- GOVSEC: Governance, compliance, regulations, security policies, audits\n"
        "- DATA: Data management, analytics, privacy, data quality\n"
        "- ECO: Digital ecosystem, partnerships, innovation, digital transformation\n"
        "- OTHER: None of the above\n\n"
        "Output JSON: {\"Topic 0: keywords\": \"DIMENSION\", ...}\n"
        "Output ONLY valid JSON, no explanation."
    )
    
    response = client.chat.completions.create(
        model=LLM_MODEL,
        temperature=LLM_TEMPERATURE,
        messages=[
            {"role": "system", "content": "You are a research assistant. Output JSON only."},
            {"role": "user", "content": mapping_prompt},
            {"role": "user", "content": json.dumps({
                "topics": list(topic_descriptions.values())
            }, ensure_ascii=False)}
        ]
    )
    
    raw = response.choices[0].message.content
    try:
        mapping_result = json.loads(raw)
    except Exception:
        match = re.search(r'\{[\s\S]*\}', raw)
        mapping_result = json.loads(match.group(0)) if match else {}
    
    # 解析映射結果（key 可能是 "Topic X: ..." 格式）
    topic_to_dimension = {}
    for key, dim in mapping_result.items():
        # 提取 topic id
        match = re.search(r'Topic (\d+)', key)
        if match:
            tid = int(match.group(1))
            topic_to_dimension[tid] = dim
    
    # 保存緩存
    with open(PHASE4_TOPIC_DIM_MAP_CACHE, 'w') as f:
        json.dump(topic_to_dimension, f, indent=2, ensure_ascii=False)
    
    print(f"  - 已保存緩存: {PHASE4_TOPIC_DIM_MAP_CACHE}")

# 統計映射結果
dim_counts = pd.Series(topic_to_dimension.values()).value_counts()
print("\n映射統計:")
for dim, count in dim_counts.items():
    print(f"  - {dim}: {count} 個主題")

# 顯示部分映射示例
print("\n映射示例（前10個）:")
for tid in sorted(topic_to_dimension.keys())[:10]:
    words = ', '.join([w for w, _ in model.get_topic(tid)[:5]])
    dim = topic_to_dimension[tid]
    print(f"  Topic {tid} ({words}...) → {dim}")

⚙ 映射主題到數位韌性構面...
  - 生成新映射（請求 LLM）...
  - 已保存緩存: data/phase4_topic_dimension_map.json

映射統計:
  - OTHER: 51 個主題
  - GOVSEC: 35 個主題
  - ITC: 17 個主題
  - DATA: 10 個主題
  - ACAP: 2 個主題
  - ECO: 2 個主題

映射示例（前10個）:
  Topic 0 (tax, income, taxes, deferred, foreign...) → GOVSEC
  Topic 1 (health, care, medicare, medical, unitedhealthcare...) → OTHER
  Topic 2 (products, product, healthcare, drug, generic...) → OTHER
  Topic 3 (pension, plans, benefit, plan, postretirement...) → OTHER
  Topic 4 (exxonmobil, gas, oil, exxonmobils, energy...) → OTHER
  Topic 5 (procter, gamble, net, care, sales...) → OTHER
  Topic 6 (standard, asu, adoption, accounting, update...) → GOVSEC
  Topic 7 (cash, billion, financing, activities, net...) → GOVSEC
  Topic 8 (hedges, derivative, derivatives, hedge, instruments...) → OTHER
  Topic 9 (products, earnings, industrial, revenues, manufacturing...) → OTHER


In [11]:
# ========================================
# Step 2: 主題層級評分（核心優化）
# ========================================

print("\n⚙ 對主題×構面進行評分...")

# 檢查緩存
if PHASE4_TOPIC_SCORES_CACHE.exists():
    print("  - 從緩存載入評分...")
    with open(PHASE4_TOPIC_SCORES_CACHE, 'r') as f:
        topic_scores = json.load(f)
    # 轉換 key
    topic_scores = {int(k): v for k, v in topic_scores.items()}
else:
    print("  - 生成新評分（這將大幅快於文檔級評分）...")
    
    topic_scores = {}  # {topic_id: {dim: score}}
    
    # 為每個主題生成代表性描述
    topic_ids = sorted([int(t) for t in df[TOPIC_COL].dropna().unique() if t != -1])
    
    for tid in tqdm(topic_ids, desc="評分主題"):
        # 獲取主題信息
        words = ', '.join([w for w, _ in model.get_topic(tid)[:10]])
        examples = df[df[TOPIC_COL] == tid]['text'].head(3).tolist()
        
        # 構建主題描述
        topic_desc = (
            f"Topic {tid}\n"
            f"Keywords: {words}\n"
            f"Example excerpts:\n" +
            "\n---\n".join([ex[:500] for ex in examples])
        )
        
        # 對每個構面評分
        scores = {}
        for dim in DIMENSIONS:
            scoring_prompt = (
                f"Rate this topic's relevance and substantiveness to the '{dim}' dimension "
                f"of digital resilience.\n\n"
                f"{SCORING_RUBRIC}\n\n"
                f"Dimension: {dim}\n"
                f"Topic information:\n{topic_desc}"
            )
            
            try:
                response = client.chat.completions.create(
                    model=LLM_MODEL,
                    temperature=LLM_TEMPERATURE,
                    messages=[
                        {"role": "system", "content": "You are a domain expert evaluating topics. Output JSON only."},
                        {"role": "user", "content": scoring_prompt}
                    ]
                )
                
                raw = response.choices[0].message.content
                result = json.loads(raw) if raw.startswith('{') else json.loads(re.search(r'\{[\s\S]*\}', raw).group(0))
                score = float(result.get('score', 0))
                score = max(0, min(5, score))  # 限制在 0-5
                scores[dim] = score
            except Exception as e:
                print(f"  ⚠ Topic {tid}, Dim {dim} 評分失敗: {e}")
                scores[dim] = 0.0
        
        topic_scores[tid] = scores
    
    # 保存緩存
    with open(PHASE4_TOPIC_SCORES_CACHE, 'w') as f:
        json.dump(topic_scores, f, indent=2, ensure_ascii=False)
    
    print(f"  - 已保存緩存: {PHASE4_TOPIC_SCORES_CACHE}")

print(f"\n✓ 完成 {len(topic_scores)} 個主題的評分")

# 顯示評分示例
print("\n評分示例（前5個主題）:")
for tid in sorted(topic_scores.keys())[:5]:
    words = ', '.join([w for w, _ in model.get_topic(tid)[:3]])
    scores = topic_scores[tid]
    print(f"  Topic {tid} ({words}...):")
    for dim, score in scores.items():
        if score > 0:
            print(f"    {dim}: {score:.1f}")


⚙ 對主題×構面進行評分...
  - 生成新評分（這將大幅快於文檔級評分）...


評分主題: 100%|██████████| 117/117 [1:34:11<00:00, 48.31s/it]

  - 已保存緩存: data/phase4_topic_dimension_scores.json

✓ 完成 117 個主題的評分

評分示例（前5個主題）:
  Topic 0 (tax, income, taxes...):
    ACAP: 2.0
    DC: 1.0
    DATA: 2.0
    ECO: 4.0
    OTHER: 2.0
  Topic 1 (health, care, medicare...):
    ITC: 2.0
    ACAP: 3.0
    DC: 3.0
    GOVSEC: 1.0
    DATA: 1.0
    ECO: 3.0
    OTHER: 2.0
  Topic 2 (products, product, healthcare...):
    ITC: 2.0
    ACAP: 2.0
    DC: 2.0
    GOVSEC: 2.0
    DATA: 2.0
    ECO: 3.0
    OTHER: 2.0
  Topic 3 (pension, plans, benefit...):
    ACAP: 3.0
    GOVSEC: 1.0
    DATA: 3.0
    ECO: 2.0
    OTHER: 2.0
  Topic 4 (exxonmobil, gas, oil...):
    ITC: 3.0
    ACAP: 3.0
    DC: 1.0
    GOVSEC: 2.0
    DATA: 3.0
    ECO: 5.0
    OTHER: 3.0


In [12]:
# ========================================
# Step 3: 文檔層級評分（基於主題分佈）
# ========================================

print("⚙ 計算文檔級別的構面評分...")

# 如果有主題概率分佈，使用加權平均；否則使用硬分配
if PHASE2_DOC_PROBS.exists():
    print("  - 使用主題概率分佈進行加權計算")
    probs = np.load(PHASE2_DOC_PROBS)
    use_probs = True
else:
    print("  - 使用硬主題分配")
    use_probs = False

doc_scores = []

for idx, row in tqdm(df.iterrows(), total=len(df), desc="計算文檔評分"):
    scores = {dim: 0.0 for dim in DIMENSIONS}
    
    if use_probs and idx < len(probs):
        # 基於主題概率的加權評分
        prob_dist = probs[idx]
        for tid, prob in enumerate(prob_dist):
            if tid == -1 or prob < 0.01:  # 忽略離群和低概率
                continue
            if tid in topic_scores:
                for dim in DIMENSIONS:
                    scores[dim] += prob * topic_scores[tid].get(dim, 0)
    else:
        # 基於硬分配
        tid = int(row[TOPIC_COL])
        if tid != -1 and tid in topic_scores:
            scores = topic_scores[tid].copy()
    
    doc_scores.append(scores)

# 合併到 DataFrame
scores_df = pd.DataFrame(doc_scores)
result_df = pd.concat([df.reset_index(drop=True), scores_df], axis=1)

# 保存
result_df.to_csv(PHASE4_DOC_SCORES_CSV, index=False, encoding='utf-8')

print(f"✓ 文檔評分完成")
print(f"  - 已保存: {PHASE4_DOC_SCORES_CSV}")
result_df[['text'] + DIMENSIONS].head()

⚙ 計算文檔級別的構面評分...
  - 使用主題概率分佈進行加權計算


計算文檔評分: 100%|██████████| 6233/6233 [00:00<00:00, 43295.82it/s]


✓ 文檔評分完成
  - 已保存: data/part4_doc_dimension_scores.csv


,text,ITC,ACAP,DC,GOVSEC,DATA,ECO,OTHER
0,\n\n10-K\n1\nbac-1231201710xk.htm\n10-K\nDocum...,0.567895,0.742863,0.770305,0.766579,0.943984,1.435887,0.831297
1,We routinely post and make accessible financia...,0.150651,0.175765,0.249193,0.229236,0.262637,0.443559,0.229552
2,"and in international markets, we provide a div...",0.273042,0.407353,0.242627,0.702621,0.826736,0.942030,0.318550
3,We compete with some of these competitors glob...,0.229346,0.368801,0.353451,0.288197,0.244599,0.381106,0.368801
4,None of our domestic employees are subject to ...,3.000000,3.000000,1.000000,2.000000,3.000000,5.000000,3.000000


In [13]:
# ========================================
# Step 4: 計算數位韌性指數（DRI）
# ========================================

print("⚙ 計算數位韌性指數（DRI）...")

# 偵測實體和時間欄位
entity_col = None
for col in ['company', 'firm', 'ticker']:
    if col in result_df.columns:
        entity_col = col
        break

time_col = None
for col in ['year', 'date']:
    if col in result_df.columns:
        time_col = col
        break

group_cols = [c for c in [entity_col, time_col] if c]

if not group_cols:
    print("  ⚠ 未偵測到 company/year，計算整體 DRI")
    agg = result_df[DIMENSIONS].mean().to_frame().T
else:
    print(f"  - 按 {group_cols} 聚合")
    agg = result_df[group_cols + DIMENSIONS].groupby(group_cols).mean().reset_index()

# 計算加權 DRI
print("  - 使用加權平均計算 DRI")
dri_scores = np.zeros(len(agg))
for dim in DIMENSIONS:
    weight = DIMENSION_WEIGHTS.get(dim, 0)
    dri_scores += agg[dim].values * weight

agg['DRI'] = dri_scores

# 保存
agg.to_csv(PHASE4_DRI_CSV, index=False, encoding='utf-8')

print(f"✓ DRI 計算完成")
print(f"  - 已保存: {PHASE4_DRI_CSV}")
print(f"\nDRI 統計:")
print(f"  - 平均值: {agg['DRI'].mean():.3f}")
print(f"  - 標準差: {agg['DRI'].std():.3f}")
print(f"  - 範圍: [{agg['DRI'].min():.3f}, {agg['DRI'].max():.3f}]")

agg.head(10)

⚙ 計算數位韌性指數（DRI）...
  - 按 ['ticker', 'year'] 聚合
  - 使用加權平均計算 DRI
✓ DRI 計算完成
  - 已保存: data/part4_entity_time_dri.csv

DRI 統計:
  - 平均值: 1.322
  - 標準差: 0.204
  - 範圍: [0.818, 1.585]


,ticker,year,ITC,ACAP,DC,GOVSEC,DATA,ECO,OTHER,DRI
0,AAPL,2018,0.732843,1.214491,1.199411,0.919845,1.080862,2.008562,1.397826,1.170769
1,AAPL,2019,0.647035,1.140406,1.164747,0.798240,0.950223,1.854673,1.329306,1.072671
2,AMZN,2018,0.475591,0.870687,0.633933,0.785840,0.928183,1.310896,0.873558,0.818084
3,AMZN,2019,0.484569,1.055978,0.722968,0.765328,1.108889,1.528314,1.036700,0.926934
4,BAC,2018,0.739170,1.219561,0.817715,1.259404,1.502800,2.012307,0.964983,1.230580
5,BAC,2019,0.638730,1.051745,0.688897,1.225159,1.376890,2.017759,1.142673,1.134401
6,BRK-B,2018,0.848677,1.271849,1.049523,1.524231,1.952482,2.036254,1.467172,1.408479
7,BRK-B,2019,0.903161,1.458350,1.197646,1.616142,2.092728,2.241823,1.651319,1.544553
8,CSCO,2018,1.044537,1.507539,1.043962,1.170067,1.587339,2.066058,1.480383,1.390529
9,CSCO,2019,1.002996,1.451169,1.014497,1.219425,1.624125,2.019357,1.474923,1.372443


In [14]:
# ========================================
# 可視化 DRI
# ========================================

import plotly.express as px

if entity_col and time_col:
    fig = px.line(
        agg,
        x=time_col,
        y='DRI',
        color=entity_col,
        markers=True,
        title='數位韌性指數（DRI）時序趨勢'
    )
    fig.show()
elif entity_col:
    fig = px.bar(
        agg,
        x=entity_col,
        y='DRI',
        title='各實體的數位韌性指數（DRI）'
    )
    fig.show()
else:
    fig = px.bar(
        agg,
        x=list(range(len(agg))),
        y='DRI',
        title='整體數位韌性指數（DRI）'
    )
    fig.show()

print("\n✓ 可視化完成")


✓ 可視化完成


---
# 總結

## 優化成果

### 1. 集中配置管理
- 所有關鍵參數在第一個 cell 統一管理
- 易於調整和實驗

### 2. Phase 3 多輪迭代
- 支持多輪 LLM 優化（可配置）
- 追蹤每輪的指標變化

### 3. Phase 4 主題層級評分
- **速度提升**: 從 6000+ 文檔 → ~70 主題
- **準確性提升**: LLM 獲得主題關鍵詞，不再盲目猜測
- 使用主題概率分佈加權計算文檔評分

### 4. 緩存機制
- 嵌入向量緩存
- Phase 3 優化計劃緩存
- Phase 4 主題映射和評分緩存

### 5. 加權 DRI 計算
- 支持自定義構面權重
- 更靈活的指標計算

## 輸出文件清單

- `data/part2_bertopic_model/` - Phase 2 模型
- `data/part2_topics.csv` - Phase 2 主題列表
- `data/part2_corpus_with_topics.csv` - Phase 2 帶主題標註的語料
- `data/part3_optimized_bertopic_model/` - Phase 3 優化模型
- `data/part3_corpus_with_topics_v2.csv` - Phase 3 優化後語料
- `data/phase3_optimization_plans.json` - 優化歷史記錄
- `data/phase4_topic_dimension_map.json` - 主題-構面映射緩存
- `data/phase4_topic_dimension_scores.json` - 主題評分緩存
- `data/part4_doc_dimension_scores.csv` - 文檔級構面評分
- `data/part4_entity_time_dri.csv` - 最終 DRI 指數